In [ ]:
import numpy as np
from collections import defaultdict

In [ ]:
file = "/Users/liesbeth_bogaert/Documents/AoC/2022/input/input_day17.txt"

In [ ]:
lines = open(file).read().strip().splitlines()
#print(lines[0])
input = lines[0]
#print(input[0])

PART 1: binary rocks

In [ ]:
#Rock: rows in binary + rock width
ROCKS = [
    ([15], 4),  # 0b1111
    ([2, 7, 2], 3),  # 010, 111, 010
    ([1, 1, 7], 3),  # 001, 001, 111
    ([1, 1, 1, 1], 1),  # 1, 1, 1, 1, 1
    ([3, 3], 2),  # 11, 11
]

In [ ]:
def collision(board, row, col, rock, rock_width):
    for i, r in enumerate(rock):
        if board[row + i] & (r << (7 - rock_width - col)):
            return True
    return False

In [ ]:
height = 10000
row = height
minrow = height
board = np.zeros(height, dtype=np.ubyte)

rock_id = 0
movement_id = 0

n = 2022
while n > 0:

    col = 2

    rock, rock_width = ROCKS[rock_id%len(ROCKS)]
    rock_height = len(rock)
    row = minrow - 3 - rock_height

    while True:
        movement = input[movement_id]
        if(movement_id < len(input)-1): # Repeat input
            movement_id += 1
        else:
            movement_id = 0

        #MOVE
        if (movement == ">" and col + rock_width < 7 and not collision(board, row, col+1, rock, rock_width)):
            col += 1
        elif (movement == "<" and col > 0 and not collision(board, row, col-1, rock, rock_width)):
            col -= 1
        
        #FLOOR
        if rock_height + row + 1 > height:
            break
        #COLLISION
        if collision(board, row + 1, col, rock, rock_width):
            break
        #DOWN
        row += 1


    minrow = min(row, minrow)
    for i, r in enumerate(rock):
        board[row + i] |= r << (7 - rock_width - col)
    
    rock_id += 1
    n -= 1

print(height - minrow)

PART 2: 
- restart because part one is not fast enough :(


In [ ]:
ROCKS = [
    [(2, 0), (3, 0), (4, 0), (5, 0)], # Horizontal Line
    [(3, 0), (2, 1), (3, 1), (4, 1), (3, 2)], # Plus
    [(2, 0), (3, 0), (4, 0), (4, 1), (4, 2)], # L
    [(2, 0), (2, 1), (2, 2), (2, 3)], # Vertical Bar
    [(2, 0), (2, 1), (3, 0), (3, 1)], # Square
]

In [ ]:
def solve(data, cycles):
    width = 7

    tower = {(x, 0) for x in range(width)}
    tower_height = 0
    column_heights = [0] * width

    cycle_id = 0
    mov_id = 0

    history = defaultdict(list)

    while cycle_id < cycles:
        current_cycle = cycle_id
        cycle_id += 1
        
        rock_id = current_cycle % len(ROCKS)
        rock = ROCKS[rock_id]
        y_off = tower_height + 4
        rock = [(x, y+y_off) for x, y in rock]

        cycle_key = (rock_id, mov_id)

        move_x, move_y = 0, 0

        while True:
            # move left or right (from input file)
            movement = -1 if data[mov_id] == '<' else 1
            mov_id = (mov_id + 1) % len(data)

            moved_rock = [(x+movement, y) for x, y in rock]
            if any(x < 0 or x >= width for x, y in moved_rock):
                moved_rock = rock
            elif any(c in tower for c in moved_rock):
                moved_rock = rock
            else:
                move_x += movement

            rock = moved_rock
            #move down
            moved_rock = [(x, y-1) for x,y in rock]
            
            if any(c in tower for c in moved_rock):
                tower.update(rock)
                tower_height = max(tower_height, max(y for x,y in rock))
                for x, y in rock:
                    column_heights[x] = max(column_heights[x], y)

                #needed for part 2
                if history is None:
                    break

                height_diff = tuple(y-tower_height for y in column_heights)
                key = (cycle_key, move_x, move_y, height_diff)

                cycle_history = history[key]

                if len(cycle_history) > 1:
                    last_y_diff = cycle_history[-1][0] - cycle_history[-2][0]
                    curr_y_diff = tower_height - cycle_history[-1][0]

                    if last_y_diff == curr_y_diff:
                        cycle_diff = current_cycle - cycle_history[-1][1]

                        todo = cycles - current_cycle - 1

                        supercycles = todo // cycle_diff
                        skipped_cycles = supercycles * cycle_diff

                        height_mod = supercycles * curr_y_diff

                        tower = {(x, y+height_mod) for x, y in tower}
                        tower_height += height_mod

                        cycle_idx += skipped_cycles

                        history = None
                        break

                cycle_history.append((tower_height, current_cycle))
                break

            move_y -= 1
            rock = moved_rock
        print(tower)

    return tower_height

In [ ]:
answer = solve(input, 2)
print(answer)

In [ ]:
answer = solve(input, 1000000000000)
print(answer)